In [66]:
import folium
import numpy as np
import pandas as pd
europePath="topojson/europe.topojson.json"
swissPath="topojson/ch-cantons.topojson.json"
unem_eur_data=pd.DataFrame.from_csv('une.tsv', sep='\t', header=0)
months=["Jan","Feb","Mar","Apr","May","June","July","Aug","Sep"]

<h1>Task 1<h1>

In [79]:
'''
This data goes all the way back to 1983, we just want it for 2017
TOTAL_PC represents percentage of population that is unemployed
THS_PER is how many people (in thousands) are unemployed
Metadata explanation: http://ec.europa.eu/eurostat/cache/metadata/en/une_esms.htm
We also have NSA,SA and TC (Non seasonally adjusted, seasonally adjusted and ??? )
'''
unem_eur_data.head()

#Create list that has columns from 1 to 9 (jan-sep)
col2017=[]
for i in range(1,10):
    col2017.append("2017M0"+str(i)+" ")

#Only select 2017 data
df=unem_eur_data[col2017]

#Returns rows with the specific fields
def getRows(k,keys):
    pct_rows=[]
    for k in keys:
        kFields=k.split(",")
        if((kFields[0]=="SA")&(kFields[1]=="TOTAL")&(kFields[2]=="PC_ACT")&(kFields[3]=="T")):
            pct_rows.append(k)
    return pct_rows

df=df.loc[getRows(df.index,df['2017M01 '].keys())]
df.index.name="Countries"

#http://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes
countries=["Austria","Belgium","Bulgaria","Czech Republic","Denmark","Germany","Estonia","Ireland","Greece","Spain","France",
           "Croatia","Italy","Cyprus","Latvia","Lithuania","Luxembourg","Hungary","Malta","Netherlands","Poland",
          "Portugal","Romania","Slovenia","Slovakia","Finland","Sweden","United Kingdom","Norway","Turkey","Iceland"]
codes=["AT","BE","BG","CZ","DK","DE","EE","IE","EL","ES","FR","HR","IT","CY","LV","LT","LU","HU","MT","NL","PL","PT","RO","SI",
      "SK","FI","SE","UK","NO","TR","IS"]

#Rename rows into country names
for i in range(len(countries)):
    df=df.rename(index={"SA,TOTAL,PC_ACT,T,"+codes[i]:countries[i] })

df.columns=months
df=df.replace(to_replace=":",value=0)
df=df.drop(["SA,TOTAL,PC_ACT,T,EA","SA,TOTAL,PC_ACT,T,EA18","SA,TOTAL,PC_ACT,T,EA19",
            "SA,TOTAL,PC_ACT,T,EU25","SA,TOTAL,PC_ACT,T,EU27","SA,TOTAL,PC_ACT,T,EU28",
          "SA,TOTAL,PC_ACT,T,JP", "SA,TOTAL,PC_ACT,T,US"])
df_avg=df.drop(["Aug","Sep"],axis=1)
df_avg['Jan']['Belgium']=7.6
df_avg=df_avg.astype(float)
df_avg=df_avg.mean(axis=1)
df_avg

Countries
Austria            5.557143
Belgium            7.442857
Bulgaria           6.300000
Cyprus            11.685714
Czech Republic     3.114286
Germany            3.814286
Denmark            5.857143
Estonia            6.057143
Greece            21.928571
Spain             17.628571
Finland            8.771429
France             9.600000
Croatia           11.385714
Hungary            4.314286
Ireland            6.457143
Iceland            2.828571
Italy             11.385714
Lithuania          7.528571
Luxembourg         6.042857
Latvia             8.871429
Malta              4.171429
Netherlands        5.085714
Norway             4.328571
Poland             5.071429
Portugal           9.485714
Romania            5.157143
Sweden             6.728571
Slovenia           6.885714
Slovakia           8.300000
Turkey            11.385714
United Kingdom     4.414286
dtype: float64

In [105]:
df_swi=pd.read_excel("Swiss.xlsx",encoding = "ISO-8859-1")
df_swi=df_swi.drop([0,1,2])
df_swi=df_swi.drop("Unnamed: 1",axis=1)
df_swi=df_swi.drop("Unnamed: 20",axis=1)
df_swi=df_swi.drop("Unnamed: 21",axis=1)
df_swi=df_swi.drop("Unnamed: 22",axis=1)
df_swi=df_swi.drop("Unnamed: 23",axis=1)
for i in range(2,20):
    if(i%2==1):        
        df_swi=df_swi.drop("Unnamed: "+str(i),axis=1)
df_swi=df_swi.set_index('2.1 Arbeitslosenquoten')
df_swi.columns=months
df_swi.index=df_swi.index.rename("Canton")
df_swi=df_swi.drop("Gesamt")
df_swi.sort_index(axis=0)

#German names - tessin=ticino

,Jan,Feb,Mar,Apr,May,June,July,Aug,Sep
Canton,,,,,,,,,
Aargau,3.5,3.4,3.3,3.2,3.1,3,2.9,3,2.9
Appenzell Ausserrhoden,2,1.9,1.8,1.8,1.7,1.7,1.8,1.7,1.7
Appenzell Innerrhoden,1.4,1.3,1.1,1,0.8,0.9,0.8,0.7,0.7
Basel-Landschaft,3.1,3.1,3.1,2.9,2.9,2.8,2.8,2.8,2.8
Basel-Stadt,4.2,4.1,4,3.9,3.7,3.6,3.5,3.7,3.5
Bern,3,3,2.9,2.7,2.6,2.4,2.4,2.5,2.4
Freiburg,3.2,2.9,2.7,2.7,2.4,2.4,2.7,2.8,2.7
Genf,5.7,5.5,5.4,5.3,5.2,5.1,5.1,5.2,5.2
Glarus,2.5,2.6,2.4,2.3,2.1,2,1.8,1.9,1.8


In [103]:
import json
from pprint import pprint

with open(swissPath) as data_file:    
    data = json.load(data_file)
swiss_ids=[]
for i in data['objects']['cantons']['geometries']:
    swiss_ids.append(i['id'])
sorted(swiss_ids)

#pprint(data['objects']['cantons']['geometries'][0])

['AG',
 'AI',
 'AR',
 'BE',
 'BL',
 'BS',
 'FR',
 'GE',
 'GL',
 'GR',
 'JU',
 'LU',
 'NE',
 'NW',
 'OW',
 'SG',
 'SH',
 'SO',
 'SZ',
 'TG',
 'TI',
 'UR',
 'VD',
 'VS',
 'ZG',
 'ZH']

In [162]:
#np.arange(df_avg.min(),,5)
df_avg.max()

21.928571428571434

In [169]:
eur_map = folium.Map(location=[55, 15],tiles='Mapbox Bright',zoom_start=3)

eur_map.choropleth(
    geo_data="europe.geojson",
    data=df_avg,
    fill_color='OrRd', 
    fill_opacity=0.5, 
    line_opacity=0.2,
    line_color="black",
    key_on='feature.properties.NAME',
    legend_name="Unemployment Rate in %",
    highlight=True,
    threshold_scale=[3,6.8,10.6,14.4,18.2,23]

) #Threshold scale will change. Need to have threshold go from
#min to max of df_avg
#    threshold_scale=np.arange(df_avg.min(),df_avg.max(),3)
#need darkest red to be darker
eur_map

In [50]:
swiss_map = folium.Map(location=[46.8182, 8.2275],tiles='Mapbox Bright', zoom_start=8)

folium.TopoJson(open(swissPath),
                'objects.cantons',
                name='topojson',
               ).add_to(swiss_map)

#swiss_map


folium.features.TopoJson

In [73]:
europe_map = folium.Map(location=[55, 15],tiles='Mapbox Bright', zoom_start=3)

folium.TopoJson(open(europePath),
                'objects.europe',
                name='topojson',
               ).add_to(europe_map)

europe_map